In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def docx_to_fragments(docx: Docx2txtLoader, spliter : RecursiveCharacterTextSplitter):
    docs = docx.load()
    fragment = spliter.create_documents([x.page_content for x in docs])    
    return fragment

In [3]:
loader = Docx2txtLoader(
    './dataset_input/Инструкция_пользователя_по_работе_в_системе_Naumen_Service_Desk.docx'
)

split = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128
)

In [4]:
data = docx_to_fragments(loader, split)

In [5]:
from langchain.vectorstores import FAISS
from localEmbeddings import LocalEmbeddings

In [6]:
embeddings_model = LocalEmbeddings(
    model = 'nomic-embed-text-v1.5',
    linck='http://172.20.10.7:1234/v1/embeddings'
)

In [7]:
vectorstore = FAISS.from_texts(
    [
        i.page_content.replace("\n", " ")
        for i in data
    ], 
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

In [8]:
from langchain_openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate

In [9]:
model = OpenAI(
    base_url='http://172.20.10.7:1234/v1',
    model='hermes-3-llama-3.1-8b',
    api_key='non'
)

In [10]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай краткий ответ на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = model.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [11]:
out = query(input())
print(out[0])
print(out[1])

 

AI: Для того чтобы закрыть программу, выполните следующие действия:

1. Щелкните правой кнопкой мыши на заголовке окна программы (верхняя часть окна).
2. В выпадающем меню выберите пункт «Закрыть».
3. Программа закроется.

Также можно закрыть программу, нажав сочетание клавиш `Alt + F4` на клавиатуре. Это действие позволит сразу закрыть текущее окно приложения и вернуться к рабочему столу или другому открытым приложениям.

Помимо этого, можно использовать стандартное окно задач Task Manager (нажав `Ctrl + Shift + Esc` на клавиатуре) или меню «Пуск» на панели задач (кликнув по нему слева на экране). В этих случаях также доступна возможность закрытия программы.<|end_of_text|><|begin_of_text|>://
```<|end_of_text|><|begin_of_text|>:// 
```
* 1:0:2
```<|end_of_text|><|begin_of_text|>_REF_ANSWER
Для того чтобы закрыть программу, выполните следующие действия:
1. Щелкните правой кнопкой мы
[Document(page_content='Регистрация заявки через ЛК  Для того чтобы зарегистрировать заявку через ЛК,